In [1]:
from tensorflow import keras

In [2]:
new_model = keras.models.load_model('autocomplete_model.h5')

In [5]:
import pickle

# Load the variables from the pickle file
with open('autocomplete_variables.pkl', 'rb') as f:
    loaded_variables = pickle.load(f)

# Assuming you saved the variables as a dictionary or similar structure
# You can access them like this:
word_to_index = loaded_variables['word_to_index']
index_to_word = loaded_variables['index_to_word']
max_sequence_length = loaded_variables['max_sequence_length']

print("Variables loaded successfully!")

Variables loaded successfully!


In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_suggestion(model, tokenizer, max_sequence_length, seed_text, n_words=1):
    """
    Genera la siguiente palabra en una secuencia de texto.

    Args:
        model: El modelo Keras entrenado.
        tokenizer: El tokenizador utilizado para preprocesar los datos.
        max_sequence_length: La longitud máxima de las secuencias de entrada.
        seed_text: El texto inicial para generar la sugerencia.
        n_words: El número de palabras a generar (por defecto es 1).

    Returns:
        La secuencia de texto con la palabra predicha añadida.
    """
    for _ in range(n_words):
        # Preprocesar el texto semilla
        token_list = [tokenizer.word_to_index[word] for word in seed_text.split() if word in tokenizer.word_to_index]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length, padding='pre')

        # Predecir la siguiente palabra
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted_probs, axis=-1)[0]

        # Convertir el índice a palabra
        predicted_word = tokenizer.index_to_word.get(predicted_word_index, "") # Use .get for safer lookup

        # Añadir la palabra predicha al texto semilla
        seed_text += " " + predicted_word
    return seed_text

# Nota: Necesitamos crear un "tokenizer" que mapee palabras a índices y viceversa.
# Usaremos los diccionarios word_to_index y index_to_word que ya creamos.
# Podemos crear una clase simple o usar los diccionarios directamente en la función.
# Para simplificar, adaptaremos la función para usar los diccionarios existentes.

In [6]:
# Adaptar la función de generación para usar los diccionarios existentes
def generate_suggestion_adapted(model, word_to_index, index_to_word, max_sequence_length, seed_text, n_words=1):
    """
    Genera la siguiente palabra en una secuencia de texto utilizando diccionarios existentes.

    Args:
        model: El modelo Keras entrenado.
        word_to_index: Diccionario que mapea palabras a índices.
        index_to_word: Diccionario que mapea índices a palabras.
        max_sequence_length: La longitud máxima de las secuencias de entrada.
        seed_text: El texto inicial para generar la sugerencia.
        n_words: El número de palabras a generar (por defecto es 1).

    Returns:
        La secuencia de texto con la palabra predicha añadida.
    """
    for _ in range(n_words):
        # Preprocesar el texto semilla
        token_list = [word_to_index[word] for word in seed_text.lower().split() if word.lower() in word_to_index]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length, padding='pre')

        # Predecir la siguiente palabra
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted_probs, axis=-1)[0]

        # Convertir el índice a palabra
        predicted_word = index_to_word.get(predicted_word_index, "")

        # Añadir la palabra predicha al texto semilla
        seed_text += " " + predicted_word
    return seed_text


# Solicitar entrada al usuario
user_input = input("Introduce un fragmento inicial: ")

# Generar la sugerencia
completed_sentence = generate_suggestion_adapted(new_model, word_to_index, index_to_word, max_sequence_length, user_input, n_words=5) # Generar 5 palabras como ejemplo

# Mostrar el resultado completo
print("Sugerencia completa:", completed_sentence)

Introduce un fragmento inicial: How to get a job as sel
Sugerencia completa: How to get a job as sel a of canada coast company
